## COMP 569 - Harpreet Kaur & Soltan Soltanli

<hr/>

### 1) Importing libraries & Defining constants

In [1]:
import random
import numpy as np

AI = 'R' # Maximizer
PLAYER = 'Y' # Minimizer
EMPTY = 0
PLAYER_PIECE = 1
AI_PIECE = 2

### 2) Generating Grid

In [2]:
def generate_random_connect_4_grid(has_played):
    rows, cols = 6, 7
    grid = [[' ' for _ in range(cols)] for _ in range(rows)]

    if has_played:
        # Randomly set the starting player
        current_player = random.choice(['R', 'Y'])

        # Making partially played game (with No Winner)
        for col in range(cols):
            num_filled = random.randint(0, rows)  # Change to allow full column filling
            for row in range(rows - num_filled, rows):
                grid[row][col] = current_player

                # Switching player with turn
                current_player = 'Y' if current_player == 'R' else 'R'

    return grid

has_played = True  # Empty Game (False), Partially Played Game (True)

### 3) Format Integration


In [3]:
# Converting the grid into the Minimax algorithm format
def convert_grid_to_minimax_format(grid):
    board = np.zeros((6, 7))
    for r in range(6):
        for c in range(7):
            if grid[r][c] == PLAYER:
                board[r][c] = PLAYER_PIECE
            elif grid[r][c] == AI:
                board[r][c] = AI_PIECE
    return board

### 4) Printing board to look better visually

In [4]:
def print_board(board):
    print(' ---------------')
    for row in range(6):
        print('|', end='')
        for col in range(7):
            if board[row][col] == PLAYER_PIECE:
                print(' Y', end='')
            elif board[row][col] == AI_PIECE:
                print(' R', end='')
            else:
                print(' .', end='')
        print(' |')
    print(' ---------------')

### 5) Detect Winner

In [5]:
def piece_to_char(piece):
    if piece == PLAYER_PIECE:
        return 'Y'
    elif piece == AI_PIECE:
        return 'R'
    else:
        return ' '

def check_horizontal(board):
    for r in range(6):
        for c in range(4):
            if board[r][c] != EMPTY and board[r][c] == board[r][c+1] == board[r][c+2] == board[r][c+3]:
                return piece_to_char(board[r][c])
    return None

def check_vertical(board):
    for r in range(3):
        for c in range(7):
            if board[r][c] != EMPTY and board[r][c] == board[r+1][c] == board[r+2][c] == board[r+3][c]:
                return piece_to_char(board[r][c])
    return None

def check_diagonal(board):
    # Bottom-Left to Top-Right
    for r in range(3):
        for c in range(4):
            if board[r][c] != EMPTY and board[r][c] == board[r+1][c+1] == board[r+2][c+2] == board[r+3][c+3]:
                return piece_to_char(board[r][c])

    # Top-Left to Bottom-Right
    for r in range(3, 6):
        for c in range(4):
            if board[r][c] != EMPTY and board[r][c] == board[r-1][c+1] == board[r-2][c+2] == board[r-3][c+3]:
                return piece_to_char(board[r][c])

    return None

def check_winner(board):
    return check_horizontal(board) or check_vertical(board) or check_diagonal(board)

### 6) Valid Location Utils

In [6]:
# Checking if a column is valid to move
def is_valid_location(board, col):
    return board[0][col] == EMPTY

# Getting valid columns to move
def get_valid_locations(board):
    valid_locations = []
    for col in range(7):
        if is_valid_location(board, col):
            valid_locations.append(col)
    return valid_locations

# Get next available row
def get_next_open_row(board, col):
    for r in range(5, -1, -1):
        if board[r][col] == EMPTY:
            return r
    return None # Column is full

### 7) Node Utils

In [7]:
# Drop piece into the board
def drop_piece_minimax(board, row, col, piece):
    board[row][col] = piece

# Check if the board is full
def is_terminal_node(board):
    return winning_move(board, PLAYER_PIECE) or winning_move(board, AI_PIECE) or len(get_valid_locations(board)) == 0

### 8) Winning move check

In [8]:
def winning_move(board, piece):
    # Check horizontal locations
    for c in range(4):
        for r in range(6):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return True
    # Check vertical locations
    for c in range(7):
        for r in range(3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return True
    # Check positive diagonal
    for c in range(4):
        for r in range(3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return True
    # Check negative diagonal
    for c in range(4):
        for r in range(3, 6):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return True
    return False

### 9) Heuristic function to evaluate board (Scoring)

In [9]:
def score_position(board, piece):
    score = 0

    # Score center column
    center_array = [int(i) for i in list(board[:, 3])]
    center_count = center_array.count(piece)
    score += center_count * 3

    # Score Horizontal
    for r in range(6):
        row_array = [int(i) for i in list(board[r,:])]
        for c in range(4):
            window = row_array[c:c+4]
            score += evaluate_window(window, piece)

    # Score Vertical
    for c in range(7):
        col_array = [int(i) for i in list(board[:,c])]
        for r in range(3):
            window = col_array[r:r+4]
            score += evaluate_window(window, piece)

    # Score positive sloped diagonal
    for r in range(3):
        for c in range(4):
            window = [board[r+i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)

    # Score negative sloped diagonal
    for r in range(3):
        for c in range(4):
            window = [board[r+3-i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)

    return score

### 10) Evaluating Window

In [10]:
def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER_PIECE if piece == AI_PIECE else AI_PIECE

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

### 11) MiniMax Algorithm

In [11]:
# With Alpha-Beta pruning
def minimax(board, depth, alpha, beta, maximizingPlayer):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI_PIECE):
                return (None, 1000)
            elif winning_move(board, PLAYER_PIECE):
                return (None, -1000)
            else: # Game is over, no more valid moves
                return (None, 0)
        else:
            return (None, score_position(board, AI_PIECE))

    if maximizingPlayer:
        value = -np.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece_minimax(b_copy, row, col, AI_PIECE)
            if winning_move(b_copy, AI_PIECE):
                return col, 1000
            new_score = minimax(b_copy, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                best_col = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return best_col, value
    else:
        value = np.inf
        best_col = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece_minimax(b_copy, row, col, PLAYER_PIECE)
            if winning_move(b_copy, PLAYER_PIECE):
                return col, -1000
            new_score = minimax(b_copy, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                best_col = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return best_col, value

### 12) AI Move

In [12]:
def make_ai_move(board, depth=4):
    if np.all(board != EMPTY):
        print("The board is full. No move possible.")
        return False

    col, minimax_score = minimax(board, depth, -np.inf, np.inf, True)
    print(f"Minimax suggests column: {col+1}, with score: {minimax_score}")

    if not isinstance(col, (int, np.integer)):
        print("Invalid column returned by minimax:", col)
        return False

    if not is_valid_location(board, col):
        print(f"Column {col+1} is not a valid location.")
        return False

    row = get_next_open_row(board, col)
    if row is None:
        print(f"Column {col+1} is full. This shouldn't happen if is_valid_location is correct.")
        return False

    drop_piece_minimax(board, row, col, AI_PIECE)
    print(f"\nAI chose column {col + 1}")
    return True

### 13) Execution Time

In [13]:
# Simulate the Connect Four game
def play_connect_four():
    # Generate a random grid (you can set `has_played=False` to start with an empty grid)
    grid = generate_random_connect_4_grid(has_played=True)
    # print("Initial Grid:")
    # for row in grid:
    #    print(row)

    # Convert the grid into the minimax board format (2D numpy array)
    board = convert_grid_to_minimax_format(grid)
    # print(board)

    print("\nInitial Board State:")
    print_board(board)

    # Check if there's already a winner in the initial grid
    winner = check_winner(board)
    if winner:
        print(f"\nWinner already exists: {winner}.\nNow we'll re-generate game!")
        play_connect_four()
    else:
        print("\nNo winner yet. AI's move...")
        # Make the AI move using Minimax
        if make_ai_move(board):
            # Print the board after the AI move
            print("\nBoard after AI's move:")
            print_board(board)
        else:
            print("AI couldn't make a move. The game might be over.")

# Run the game
play_connect_four()


Initial Board State:
 ---------------
| . . . R . . . |
| . . Y Y . R . |
| . . R R . Y . |
| . . Y Y . R . |
| . . R R R Y . |
| . R Y Y Y R Y |
 ---------------

No winner yet. AI's move...
Minimax suggests column: 2, with score: 1000

AI chose column 2

Board after AI's move:
 ---------------
| . . . R . . . |
| . . Y Y . R . |
| . . R R . Y . |
| . . Y Y . R . |
| . R R R R Y . |
| . R Y Y Y R Y |
 ---------------
